In [ ]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

In [ ]:
try:
    __confing.execute_notebook('notebooks/data_extract.ipynb')
except:
    pass

In [ ]:
import pandas as pd
import numpy as np

from src import value_filler as vf
from src import feature_selection as fs
from src import data_procces as dp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from src import evaluation

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix
from sklearn.model_selection import KFold

In [ ]:
def feature_importance(model, X: pd.DataFrame, y: pd.Series, n: int):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(X.columns)
    
    target_name = y.name  # Obtener el nombre de la columna objetivo desde la Serie y
    
    if target_name in feature_names_original:
        feature_names_original.remove(target_name)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return nueva_lista


In [ ]:
n='a'
df=pd.read_csv(r'data\custom\all_waves.csv')
target=f'p{n}hosp1y'

In [ ]:
selected = ['pahosp1y','paurina2y','pacholst','pabreast','paprost','papapsm','pamammog','pacage','padrinkcr','padrinkbd','padrinkb','paheight','paweight','pabmi','papaina','papainlv','papainfr','pafatigue','pawheeze','pabreath_m','paswell','pahearaid','pahearing','pasight','paglasses','palunglmt_m','pahrtatlmt','pastroklmt','paarthlmt','palowermoba','pauppermoba','pagrossaa','palgmusaa','pamobilaa','paclims','pasit','paarms','palift','pastoop','pachair','pameals','pashop','pamoney','pameds','pabedhlp','pabed','patoilt','paeat','pabath','pawalkr','padress','pahipcomp','pafall','pafallinj','pamhip','pahipe_m','parxarthr','parxstrok','parxhrtat','pacncrothr','pacncrmeds','pacncrradn','pacncrsurg','pacncrchem','parxdiab','parxdiabo','parxhibp','parechrtatt','parifaany','parfaany','parfcaren','pararcare','padresshlp', 'pawalkhlp', 'pabathehlp', 'paeathlp', 'pabedhlp', 'patoilethlp', 'pamealhlp', 'pashophlp', 'pamedhlp', 'pamoneyhlp','pahibpe','pagender','padiabe','pacancre','palunglmt_m','parxlung_m','pahrtatte','pastroke','paarthre','parifaany','parafaany','pahigov','pasmokev','pasmoken','paoangry','paosleep','paodngr','paodngr','paopace','paoplot','paoalchl','pawthh','paagey','pamomage','padadage','paprmem','parjudg','parorgnz']

In [ ]:
selected = list(set(selected))
df = df.dropna(subset=[target]) 
df = df[selected].copy()

In [ ]:
porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (porcentaje_nulos <= 0.4)]

In [ ]:
# fill
y = df[target]
X = df.drop(target,axis=1)

In [ ]:
from src import feature_selection as fs
X = fs.fast_fill(X)

In [ ]:
# split
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=40)

In [ ]:
# # resample 
X_train, y_train = dp.apply_resample(X_train, y_train,v=2)

In [ ]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [ ]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f'AUC={auc}')
print(y_test.value_counts())

In [ ]:
print(feature_importance(model, X, y, 65))

In [ ]:
print(feature_importance(model, X, y, 70))

In [ ]:
print(feature_importance(model, X, y, 75))

In [ ]:
f80=feature_importance(model, X, y, 80)
print(feature_importance(model, X, y, 80))

In [ ]:
raise ValueError('error')

In [ ]:
def procesar_dataframe(df, columnas, name):
    df_suma = df[columnas].sum(axis=1)
    df = df.drop(columns=columnas)
    df[name] = np.where(df_suma != 0, 1, np.nan)
    return df[name]

In [ ]:
f80=['pagrossaa', 'parifaany', 'pashop', 'pacholst', 'palift', 'pafatigue', 'pabreath_m', 'parxhibp', 'parfcaren', 'paswell', 'padrinkcr', 'paurina2y', 'parxdiabo', 'pararcare', 'pahrtatte', 'pawalkr', 'pafall', 'parxdiab', 'pahigov', 'pafallinj', 'pahipcomp', 'pabathehlp', 'pacncrchem', 'parxhrtat', 'padiabe', 'pahipe_m', 'paprost', 'pacancre', 'parxlung_m', 'pagender', 'pauppermoba', 'pacncrsurg', 'parfaany', 'paopace', 'pacncrradn', 'parxarthr', 'patoilt', 'padrinkb', 'pabath', 'padrinkbd', 'pachair', 'palunglmt_m', 'pastroke', 'papaina', 'pamammog', 'pasmokev', 'parafaany', 'pastroklmt', 'pabedhlp', 'pawheeze', 'pabed', 'papapsm', 'pahrtatlmt', 'pacage', 'pameals', 'paodngr', 'palowermoba', 'palgmusaa', 'pahearaid', 'paeat', 'pahibpe', 'pasit', 'pacncrmeds', 'pabmi', 'pameds', 'parechrtatt', 'paarthlmt', 'pabreast', 'pamobilaa', 'paweight', 'paglasses', 'padresshlp', 'paclims', 'paagey', 'papainfr', 'pahearing', 'pasmoken', 'padadage', 'pamomage', 'pawthh']

In [ ]:
var=vf.find_column_name(df,'dad')
b,nb=fs.get_bin_columns(df[var])
print(b)

### help

In [ ]:
padress=['padressa', 'padresshlp']
df2['padress']=procesar_dataframe(df, padress,'padress')

In [ ]:
shop=['pashopa', 'pashophlp']
df2['pashop']=procesar_dataframe(df, shop,'pashop')

In [ ]:
pabed=['pabeda', 'pabedhlp', 'pabede']
df2['pabed']=procesar_dataframe(df, pabed,'pabed')

In [ ]:
pameal=['pamealsa', 'pamealhlp']
df2['pameal']=procesar_dataframe(df, pameal,'pameal')

In [ ]:
pamed=['pamedsa', 'pacncrmeds', 'pamedhlp']
df2['pamed']=procesar_dataframe(df, pamed,'pamed')

In [ ]:
pabath=['pabatha', 'pabathehlp']
df2['pabath']=procesar_dataframe(df, pabath,'pabath')

In [ ]:
patoilt=['patoilta', 'patoilethlp']
df2['patoilt']=procesar_dataframe(df, patoilt,'patoilt')

In [ ]:
eat=['paeat', 'paeathlp']
df2=procesar_dataframe(df, eat,'paeat')

In [ ]:
pamobil=['pamobilaa', 'pamobilseva']
df2=procesar_dataframe(df, pamobil,'pamobil')

In [ ]:
pawalk=['pawalkra', 'pawalksa', 'pawalk1a', 'pawalkhlp', 'pawalkre']
df2=procesar_dataframe(df, pawalk,'pawalk')

### diases

In [ ]:
pahrtat=['pahrtatte', 'parxhrtat', 'pahrtatlmt']
df2['pahrtat']=procesar_dataframe(df, pahrtat,'pahrtat')

In [ ]:
pastrok=['pastroke', 'parxstrok', 'pastroklmt']
df2['pastrok']=procesar_dataframe(df, pastrok,'pastrok')

In [ ]:
parth=['paarthre', 'parxarthr', 'paarthlmt']
df2['parth']=procesar_dataframe(df, parth,'parth')

In [ ]:
pacnc=['pacncrchem', 'pacncrsurg', 'pacncrradn', 'pacncrmeds', 'pacncrothr']
df2['pacnc']=procesar_dataframe(df, pacnc,'pacnc')

In [ ]:
palung=['palunge_m', 'parxlung_m', 'palunglmt_m']
df2['palung']=procesar_dataframe(df, palung,'palung')

In [ ]:
padiab=['padiabe', 'parxdiabo', 'parxdiabi', 'parxdiab']
df2['padiab']=procesar_dataframe(df, padiab,'padiab')

### care

In [ ]:
pacare=['parascare', 'paraccare', 'pararcare', 'parafcare', 'parapfcare', 'parapfcarehrm', 'pariscare', 'pariccare', 'parircare', 'parircaredpmm', 'parifcare', 'parifcaredpmm', 'parifcarehrm', 'paripfcare', 'paripfcaredpmm', 'paripfcarehrm', 'parscare', 'parccare', 'parrcare', 'parfcare', 'parpfcare', 'parpfcaredpmm', 'parpfcarehrm', 'pagpcare']
df2['pacare']=procesar_dataframe(df, pacare,'pacare')

In [ ]:
parfaany=['paracaany', 'parafaany', 'paricaany', 'parifaany', 'parcaany', 'parfaany']
df2['parfaany']=procesar_dataframe(df, parfaany,'parfaany')

### bicios

In [ ]:
pasmoke=['pasmokev', 'pasmoken']
df2['pasmoke']=procesar_dataframe(df, pasmoke,'pasmoke')

In [ ]:
padrink=['padrink', 'padrinkb', 'padrinkcut', 'padrinkcr', 'padrinkbd', 'padrinknr']
df2['padrink']=procesar_dataframe(df, padrink,'padrink')

### inj

In [ ]:
pafall=['pafall', 'pafallinj']
df2['pafall']=procesar_dataframe(df, pafall,'pafall')

In [ ]:
pahip=['pahipe_m', 'pahipriv', 'pahipcomp', 'pahipsft', 'pahiptryu', 'pahipref']
df2['pahip']=procesar_dataframe(df, pahip,'pahip')

In [ ]:
papain=['papainfr', 'papaina']
df2['papain']=procesar_dataframe(df, papain,'papain')

### independiente

In [ ]:
len(['palift','pawthh','pamomage','padadage','pahearing','paagey','paclims','parechrtatt','paglasses','paweight','pabreast','pasita','pabmi','pagrossaa','pabreath_m','paodngr','pahearaid','palgmusaa','palowermoba','pacage','papapsm','pacholst','pabreath_m','pafatigue','paswell','paurina2y','pahrtatte','pahigov','pacncrchem','paprost','pacancre','pagender','pauppermoba','paopace','pachaira','pamammog','pawheeze'])